# ArboLab Local Workflow Example

This notebook demonstrates how to use ArboLab locally in a pure Python environment. 
It covers:
1. Imports
2. Lab Initialization
3. Entity Management (CRUD)
4. Querying and Exploration

## 1. Imports and Setup

First, we set up logging and import the central `Lab`. We also perform cleanup to ensure a fresh run (preserving inputs).

In [ ]:
import shutil
from datetime import datetime
from pathlib import Path

from arbolab.core.security import LabRole
from arbolab.lab import Lab
from arbolab.models.core import Project, Tree
from sqlalchemy import select
from arbolab_logger import LoggerConfig, configure_logger, get_logger

# Clean up previous run (delete workspace/results, keep input)
base_path = Path("./example_workspace").resolve()
shutil.rmtree(base_path / "workspace", ignore_errors=True)
shutil.rmtree(base_path / "results", ignore_errors=True)

# Ensure input exists
(base_path / "input").mkdir(parents=True, exist_ok=True)

# Configure Logging (ArboLab will automatically add file logging upon initialization)
configure_logger(LoggerConfig(
    level="DEBUG",
    colorize=True, 
))

<Logger arbolab (DEBUG)>

## 2. Lab Initialization

We initialize a Lab by specifying a `base_root`. ArboLab will automatically create a clean structure:
- `workspace/`: For the database and config
- `input/`: For raw data ingestion
- `results/`: For analysis outputs

In [13]:
# Open the Lab using base_root to enforce standard structure
# Note: workspace_root must be explicitly None when using base_root due to API signature
lab = Lab.open(workspace_root=None, base_root=base_path, role=LabRole.ADMIN)

[2026-01-18 12:55:52] INFO     Creating new workspace directory at                                       ]8;id=307167;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=841631;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#124\124]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exampl           
                               e_workspace/workspace                                                               

[2026-01-18 12:55:52] INFO     Created default config at                                              ]8;id=422078;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/config.py\config.py]8;;\:]8;id=186110;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/config.py#137\137]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exa              
                               mple_workspace/workspace/config.yaml                                                

[2026-01-18 12:55:52] INFO     Creating database directory:                                            ]8;id=888715;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/layout.py\layout.py]8;;\:]8;id=113603;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/layout.py#47\47]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exam             
                               ple_workspace/workspace/db                                                          

[2026-01-18 12:55:53] INFO     Lab initialized at                                                         ]8;id=390221;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=610091;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#80\80]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/example          
                               _workspace/workspace (Role: LabRole.ADMIN)                                          

## 3. Creating and Managing Entities

ArboLab uses a "Recipe-First" approach. Every modification is recorded, ensuring reproducibility. 
You can use `define_*`, `modify_*`, and `remove_*` methods directly on the `lab` object.

In [6]:
# 3.1 Create a Project
project = lab.define_project(
    name="Forest Monitoring 2026",
    description="Analyzing tree growth and health in the local test area."
)
print(f"Created Project: {project.name} (ID: {project.id})")

# 3.2 Create an Experiment within the project
experiment = lab.define_experiment(
    project_id=project.id,
    name="Spring Growth Campaign",
    start_time=datetime.now(),
    description="Initial measurements for the spring season."
)
print(f"Created Experiment: {experiment.name} (ID: {experiment.id})")

# 3.3 Create a 'Thing' (e.g., a Tree)
# First define the base 'Thing'
thing = lab.define_thing(
    project_id=project.id,
    name="Oak-01",
    kind="tree"
)

# Then add specific Tree metadata
tree = lab.define_tree(
    id=thing.id, # Must match the Thing ID
    height=1500, # in cm
    circumference=120
)
print(f"Created Tree with Thing ID: {thing.id}")

### Modifying Entities
Updating data is just as easy. Changes are also recorded in the workspace recipe.

In [7]:
# Update the project description
lab.modify_project(
    id=project.id, 
    description="Updated: Now including more sensors."
)

# Update tree height
lab.modify_tree(
    id=tree.id,
    height=1550
)
print("Entities updated successfully.")

## 4. Querying Data

You can access the internal database session for advanced querying using SQLAlchemy.

In [8]:
with lab.database.session() as session:
    # Query all projects
    stmt = select(Project)
    projects = session.execute(stmt).scalars().all()
    
    print(f"Found {len(projects)} projects in the Lab:")
    for p in projects:
        print(f" - {p.name} (ID: {p.id})")

    # Query trees with height > 1000
    stmt = select(Tree).where(Tree.height > 1000)
    big_trees = session.execute(stmt).scalars().all()
    print(f"\nTrees taller than 10m: {len(big_trees)}")

## 5. What else can ArboLab do?

- **Recipes**: All your actions are stored in `current.json`. You can share this recipe to reproduce your exact workspace state elsewhere.
- **Plugins**: Discover and use device plugins (like linescale or treeqinetic).
- **Data Storage**: Local Parquet/DuckDB storage for high-performance time-series data.